In [56]:

import sys
sys.path.append("../channel")
from AWGN import _AWGN
import numpy as np
import scipy
from scipy import sparse 
import copy
import ray

ch=_AWGN()

#if __name__=="__main__":
    #ray.init()

In [2]:
class coding():
    def __init__(self):
        self.N=10
        self.K=5
        #self.K=self.N*(self.Wr-self.Wc)//self.Wr
        self.Wc=3
        self.Wr=6


        #prepere constants
        self.R=self.K/self.N
        self.H=self.generate_H()
        self.tG=self.HtotG()
        self.H=sparse.csr_matrix(self.H)

    #interleave N sequence
    @staticmethod
    def interleave(N):
        interleaver_sequence=np.arange(N)
        np.random.shuffle(interleaver_sequence)
        return interleaver_sequence

    def generate_H(self):
        '''
        generate regular parity check matrix
        -----------
        Wr : row weight
        Wc : column weight
        N : length of codeword 
        '''

        if self.N*self.Wc%self.Wr!=0:
            print("constant err")
            exit()

        

        #generate sub_H matrix(Wc=1)
        sub_H=np.zeros(((self.N-self.K)//self.Wc,self.N),dtype=int)
        for i in range((self.N-self.K)//self.Wc):
            sub_H[i][self.Wr*i:self.Wr*(i+1)]=1

        H=sub_H

        #generate other sub_H matrix(Wc=1)
        for i in range(self.Wc):
            sub_H2=sub_H[:,self.interleave(self.N)]
            H=np.concatenate((H,sub_H2))
        
        H=H[:self.K,:]

        return H  

In [3]:
class coding(coding):

#from https://github.com/hichamjanati/pyldpc/blob/master/pyldpc/code.py 
    @staticmethod
    def binaryproduct(X, Y):
        """Compute a matrix-matrix / vector product in Z/2Z."""
        A = X.dot(Y)
        try:
            A = A.toarray()
        except AttributeError:
            pass
        return A % 2

    @staticmethod
    def gaussjordan(X, change=0):
        """Compute the binary row reduced echelon form of X.
        Parameters
        ----------
        X: array (m, n)
        change : boolean (default, False). If True returns the inverse transform
        Returns
        -------
        if `change` == 'True':
            A: array (m, n). row reduced form of X.
            P: tranformations applied to the identity
        else:
            A: array (m, n). row reduced form of X.
        """
        A = np.copy(X)
        m, n = A.shape

        if change:
            P = np.identity(m).astype(int)

        pivot_old = -1
        for j in range(n):
            filtre_down = A[pivot_old+1:m, j]
            pivot = np.argmax(filtre_down)+pivot_old+1

            if A[pivot, j]:
                pivot_old += 1
                if pivot_old != pivot:
                    aux = np.copy(A[pivot, :])
                    A[pivot, :] = A[pivot_old, :]
                    A[pivot_old, :] = aux
                    if change:
                        aux = np.copy(P[pivot, :])
                        P[pivot, :] = P[pivot_old, :]
                        P[pivot_old, :] = aux

                for i in range(m):
                    if i != pivot_old and A[i, j]:
                        if change:
                            P[i, :] = abs(P[i, :]-P[pivot_old, :])
                        A[i, :] = abs(A[i, :]-A[pivot_old, :])

            if pivot_old == m-1:
                break

        if change:
            return A, P
        return A

    def HtotG(self,sparse=True):
        """Return the generating coding matrix G given the LDPC matrix H.
        Parameters
        ----------
        H: array (n_equations, n_code). Parity check matrix of an LDPC code with
            code length `n_code` and `n_equations` number of equations.
        sparse: (boolean, default True): if `True`, scipy.sparse format is used
            to speed up computation.
        Returns
        -------
        G.T: array (n_bits, n_code). Transposed coding matrix.
        """
        if type(self.H) == scipy.sparse.csr_matrix:
            self.H = self.H.toarray()
        n_equations, n_code = self.H.shape

        # DOUBLE GAUSS-JORDAN:

        Href_colonnes, tQ = self.gaussjordan(self.H.T, 1)

        Href_diag = self.gaussjordan(np.transpose(Href_colonnes))

        Q = tQ.T

        n_bits = n_code - Href_diag.sum()

        Y = np.zeros(shape=(n_code, n_bits)).astype(int)
        Y[n_code - n_bits:, :] = np.identity(n_bits)

        if sparse:
            Q = scipy.sparse.csr_matrix(Q)
            Y = scipy.sparse.csr_matrix(Y)

        tG = self.binaryproduct(Q, Y)

        return tG

In [4]:
cd=coding()
print(cd.tG.shape)
print(cd.H)

(10, 6)
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 2)	1
  (1, 4)	1
  (1, 5)	1
  (1, 7)	1
  (1, 9)	1
  (2, 0)	1
  (2, 2)	1
  (2, 3)	1
  (2, 4)	1
  (2, 5)	1
  (2, 8)	1
  (3, 0)	1
  (3, 1)	1
  (3, 3)	1
  (3, 4)	1
  (3, 6)	1
  (3, 8)	1


In [5]:
class encoding(coding):

    def generate_information(self):
        #generate information
        information=np.random.randint(0,2,self.K)
        return information

    def encode(self):
        information=self.generate_information()
        codeword=self.tG@information%2
        return information,codeword

In [6]:
#check
#ec=encoding()
#print(ec.H)
#_,codeword=ec.encode()
#print(codeword.shape)
#print(np.any(codeword@np.transpose(ec.H)%2!=0))

In [7]:
class decoding(coding):

    @staticmethod
    def phi(llr):
        return np.log((np.exp(llr) + 1) / (np.exp(llr) - 1))


    def ldpc_decode(self,Lc, max_iter=20):

        # initialization
        CHECK_NODE_TANH_THRES=30
        Lc=np.clip(Lc,-1*CHECK_NODE_TANH_THRES,CHECK_NODE_TANH_THRES) #clipping operaiton

        ii, jj = self.H.nonzero()
        m,n=self.H.shape
        L_mat = self.H.dot(sparse.spdiags(Lc, 0, n, n, 'csc'))
        sL_mat=sq1 = L_mat.A[ii, jj]

        # iterations
        k = 0
        success = 0
        while success == 0 and k < max_iter:
            k += 1

            # horizontal step 
            beta=phi(sL_mat)
            

            tmp=csc_matrix.tanh(L_mat).prod(axis=1) #m*1 matrix
            sparse.spdiags(beta, 0, m, m, 'csr').dot(self.H)

            # vertical step
            r0.data = np.log(r0.data.astype(complex))
            Pr0_v = np.real(np.exp(r0.sum(axis=0)))

            Pr0 = self.H.dot(sparse.spdiags(Pr0_v.ravel(), 0, n, n, 'csr'))
            sPr0 = Pr0[ii, jj].getA1()
            Q0 = np.array(sparse.csr_matrix((sPr0 * sff0, (ii, jj)), shape=(m, n)).sum(axis=0)).T

            sq0 = sPr0 * sff0 / sr0

            r1.data = np.log(r1.data.astype(complex))
            Pr1_v = np.real(np.exp(r1.sum(axis=0)))

            Pr1 = self.H.dot(sparse.spdiags(Pr1_v.ravel(), 0, n, n, 'csr'))
            sPr1 = Pr1[ii, jj].getA1()

            Q1 = np.array(sparse.csr_matrix((sPr1 * sff1, (ii, jj)), shape=(m, n)).sum(axis=0)).T
            sq1 = sPr1 * sff1 / sr1

            sqq = sq0 + sq1
            sq0 = sq0 / sqq
            sq1 = sq1 / sqq

            # tentative decoding
            QQ = Q0 + Q1
            prob = Q1 / QQ
            Q0 = Q0 / QQ
            Q1 = Q1 / QQ

            tent = (Q1 - Q0)  # soft?
            x_hat = (np.sign(tent) + 1) / 2  # hard bits estimated

            if np.all(np.fmod(self.H.dot(x_hat), 2) == 0):
                success = 1

        return x_hat.flatten(), success, k

    def decode(self,Lc):
        f0=1/(1+np.exp(-1*Lc))
        f1=1-f0
        EST_codeword,_,_=self.ldpc_decode(f0,f1)
        EST_information=EST_codeword[:self.K]
        return EST_information


In [8]:
cd=coding()

In [95]:
def phi(mat,ii,jj,m,n):
    '''
    input: 2D matrix 
    output: 2D matrix (same as H)
    '''
    smat=mat.A[ii, jj]

    smat=np.log((np.exp(smat) + 1) / (np.exp(smat) - 1))

    mat=sparse.csr_matrix((smat, (ii, jj)), shape=(m, n))

    return mat

def make_alpha(llr,ii,jj,m,n):
    '''
    input: 1D array(llr)
    output: 2D matrix (same as H)
    '''
    salpha=np.sign(llr)
    alpha=sparse.csr_matrix((salpha, (ii, jj)), shape=(m, n))

    mask=(alpha-cd.H).getnnz(axis=1)%2 #-1の数が奇数なら１、偶数なら０を出力
    mask=-2*mask+1
    #列ごとに掛け算する マイナスの列は１、プラスの列は０
    alpha=sparse.spdiags(mask, 0, m, m, 'csr').dot(cd.H)

    return alpha



def make_beta(llr,ii,jj,m,n):
    sbeta=np.abs(llr)
    beta=sparse.csr_matrix((sbeta, (ii, jj)), shape=(m, n))
    return beta


a=np.zeros(10)
Lc=ch.generate_LLR(a,0)
#print(Lc)

# initialization
#CHECK_NODE_TANH_THRES=30
#Lc=np.clip(Lc,-1*CHECK_NODE_TANH_THRES,CHECK_NODE_TANH_THRES) #clipping operaiton

ii, jj = cd.H.nonzero()
m,n=cd.H.shape
L_mat = cd.H.dot(sparse.spdiags(Lc, 0, n, n, 'csr'))
sL_mat= L_mat.A[ii, jj]


#calcurate alpha
alpha=make_alpha(sL_mat,ii,jj,m,n)
print(alpha.toarray())
#culcurate beta
beta=make_beta(sL_mat,ii,jj,m,n)
tmp=beta.sum(axis=1)
beta.data-=tmp.repeat(np.diff(cd.H.indptr))




[[-1. -1. -1. -1. -1. -1.  0.  0.  0.  0.]
 [ 0. -1.  0. -1. -1.  0.  0. -1. -1. -1.]
 [-1.  0. -1. -1.  0. -1. -1.  0.  0. -1.]
 [ 0. -1.  0. -1. -1. -1. -1.  0. -1.  0.]]


ValueError: non-broadcastable output operand with shape (24,) doesn't match the broadcast shape (1,24)

In [ ]:
#check
dc=decoding()
a=np.zeros(dc.N)
print(dc.decode(a))

TypeError: ldpc_decode() takes from 2 to 3 positional arguments but 4 were given

In [ ]:
class LDPC(coding):
        
    def LDPC(self,EbNodB):
        information,codeword=self.encode()
        Lc=ch.generate_LLR(codeword,EbNodB)
        EST_information=self.decode(Lc)
        return information,EST_information